# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import Keras and other libraries

In [3]:
import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend

Using TensorFlow backend.


## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

In [0]:
import os
os.chdir('/content/drive/My Drive/PGP-AIML/Sequence Models in NLP/Notebooks')

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [0]:
from zipfile import ZipFile
with ZipFile('data.zip', 'r') as z:
  z.extractall()

In [0]:
oscar_wilde_list = glob.glob("./data/*.txt")

In [7]:
oscar_wilde_list

['./data/For Love of the King.txt',
 './data/Salomé A tragedy in one act.txt',
 './data/Impressions of America.txt',
 './data/The Canterville Ghost.txt',
 './data/A House of Pomegranates.txt',
 './data/Miscellaneous Aphorisms_ The Soul of Man.txt',
 './data/A Woman of No Importance a play.txt',
 './data/Essays and Lectures.txt',
 './data/The Happy Prince and other tales.txt',
 './data/Rose Leaf and Apple Leaf.txt',
 './data/Vera or, The Nihilists.txt',
 './data/Lord Arthur Savile_s Crime.txt',
 './data/Poems with the Ballad of Reading Gaol.txt',
 './data/Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
 './data/Charmides and Other Poems.txt',
 './data/An Ideal Husband.txt',
 './data/The Duchess of Padua.txt',
 './data/Oscar Wilde Miscellaneous.txt',
 './data/The Ballad of Reading Gaol.txt',
 './data/Shorter Prose Pieces.txt',
 './data/Children in Prison and Other Cruelties of Prison Life.txt',
 './data/Reviews.txt',
 './data/A Critic in Pall Mall.txt',
 './data

### Read the data

Read contents of every file from the list and append the text in a new list

In [0]:
codetext = []
bookranges = []
for oscar_wilde_file in oscar_wilde_list:
    oscar_wilde_text = open(oscar_wilde_file, "r")
    start = len(codetext)
    codetext.append(oscar_wilde_text.read())
    end = len(codetext)
    bookranges.append({"start": start, "end": end, "name": oscar_wilde_file.rsplit("/", 1)[-1]})
    oscar_wilde_text.close()

## Process the text
Initialize and fit the tokenizer

In [0]:
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(codetext)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [0]:
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

Get the word count for every word and also get the total number of words.

In [0]:
#word count for every word
word_counts = tokenizer.word_counts
#total number of words
num_words = len(word_idx) + 1

Convert text to sequence of numbers

In [0]:
sequences = tokenizer.texts_to_sequences(codetext)

### Generate Features and Labels

In [0]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [0]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.7 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

This is just to check the features and labels

In [15]:
for i, sequence in enumerate(X_train[:2]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: room in algernon's flat in half moon street the room is luxuriously and artistically furnished the sound of a

Label: piano

Features: a woman of no importance a play author oscar wilde release date september 16 2014 ebook 854 this file

Label: was



## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [16]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

W0717 11:39:44.489612 140614241126272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 11:39:44.528427 140614241126272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 11:39:44.537681 140614241126272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 11:39:44.678330 140614241126272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0717 11:39:44.692908 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         3283900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32839)             2134535   
Total params: 5,464,835
Trainable params: 5,464,835
Non-trainable params: 0
_________________________________________________________________


For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [17]:
h = model.fit(X_train, y_train, epochs = 200, batch_size = 50, 
          verbose = 1)## Train the model

W0717 11:39:45.323875 140614241126272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
6510/6510 [==============================] - 22s 3ms/step - loss: 8.2417 - acc: 0.0570
Epoch 2/200
6510/6510 [==============================] - 20s 3ms/step - loss: 6.7221 - acc: 0.0616
Epoch 3/200
6510/6510 [==============================] - 20s 3ms/step - loss: 6.4449 - acc: 0.0608
Epoch 4/200
6510/6510 [==============================] - 20s 3ms/step - loss: 6.2761 - acc: 0.0602
Epoch 5/200
6510/6510 [==============================] - 20s 3ms/step - loss: 6.1197 - acc: 0.0648
Epoch 6/200
6510/6510 [==============================] - 20s 3ms/step - loss: 6.0020 - acc: 0.0677
Epoch 7/200
6510/6510 [==============================] - 20s 3ms/step - loss: 5.8519 - acc: 0.0693
Epoch 8/200
6510/6510 [==============================] - 20s 3ms/step - loss: 5.7305 - acc: 0.0763
Epoch 9/200
6510/6510 [==============================] - 20s 3ms/step - loss: 5.6324 - acc: 0.0811
Epoch 10/200
6510/6510 [==============================] - 20s 3ms/step - loss: 5.5425 - acc: 0.0810
Epoch 11/

### Save Model

In [0]:
# save the model to file
model.save('./data/model_oscar_wilde.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [0]:
# load the model
model = load_model('./data/model_oscar_wilde.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [21]:
model.fit(X_train, y_train, batch_size=50, epochs=15)

Epoch 1/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5840 - acc: 0.8189
Epoch 2/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5886 - acc: 0.8226
Epoch 3/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5676 - acc: 0.8273
Epoch 4/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5852 - acc: 0.8210
Epoch 5/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5862 - acc: 0.8177
Epoch 6/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5840 - acc: 0.8224
Epoch 7/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5665 - acc: 0.8218
Epoch 8/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5802 - acc: 0.8210
Epoch 9/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5626 - acc: 0.8273
Epoch 10/15
6510/6510 [==============================] - 20s 3ms/step - loss: 0.5462 - acc: 0.8346
Epoch 11/15
6510/65

## Evaluation

In [22]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

6510/6510 [==============================] - 6s 976us/step
[0.06200566711998652, 0.9847926231023902]

Model Performance: Log Loss and Accuracy on validation data
2790/2790 [==============================] - 3s 927us/step
[8.325727577277837, 0.37526881844339405]


## Generate text

In [23]:
seed_length=50
new_words=50
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)

        # Softmax
        preds = exp_preds / sum(exp_preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log


Original Sequence: 
pointed out because the poems had already been published in london and as he held the copyright they could not be reissued save with his consent furthermore since i have read the introduction i am not over pleased at the way in which i find myself identified with much that


Generated Sequence: 
one sure from the two bright sapphires and transcribed from the french and with the disciple life the plagiarist the indispensable east and this header and am v with the text persons in the prologue peter not those who so patiently assisted him the proof readers amongst the trial of
